# Plots and figures HCP

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import os.path as path
import os
import socket
import sys

In [ ]:
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
print("Your style sheets are located at: {}".format(path.join(mpl.__path__[0], 'mpl-data', 'stylelib')))

import pandas as pd
# import pingouin
# import scipy.stats
import seaborn as sns

%matplotlib inline

In [ ]:
sys.path.append('../../..')

from configs.configs import get_config_dict
from helpers.figures import set_size

In [ ]:
all_styles = [
    'seaborn-dark',
    'seaborn-darkgrid',
    'seaborn-ticks', 'fivethirtyeight', 'seaborn-whitegrid', 'classic',
    '_classic_test', 'fast', 'seaborn-talk', 'seaborn-dark-palette', 'seaborn-bright',
    'seaborn-pastel', 'grayscale', 'seaborn-notebook', 'ggplot', 'seaborn-colorblind',
    'seaborn-muted', 'seaborn', 'Solarize_Light2', 'seaborn-paper',
    'bmh', 'tableau-colorblind10', 'seaborn-white', 'dark_background', 'seaborn-poster', 'seaborn-deep'
]

In [ ]:
plt.style.available

In [ ]:
sns.set_style('whitegrid')
# plt.style.use('tex')

In [ ]:
# However, the figsize argument takes inputs in inches and we have the width of our document in pts. To set the figure size we construct a function to convert from pts to inches and to determine an aesthetic figure height using the golden ratio:

In [ ]:
data_dimensionality = 'd15'
data_split = 'all'
kernel_param = 'kernel_lengthscales'

n_time_series = int(data_dimensionality[1:])

In [ ]:
hcp_cfg = get_config_dict(
    data_set_name='HCP_PTN1200_recon2',
    subset_dimensionality=data_dimensionality,
    hostname=socket.gethostname()
)

### TVFC estimates

#### Fig. 6

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.plotters.plot_TVFC_estimates import _plot_tvfc_estimates

# Run this plotter on the Hivemind.

### Subject measure prediction

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.subject_measure_prediction.plot_out_of_sample_prediction_scores_TVFC_summary_measures import _plot_out_of_sample_prediction_scores, _plot_out_of_sample_prediction_scores_joint, rename_variables_for_plots, _rename_models_for_plots

In [ ]:
metric = 'correlation'
tvfc_summary_measure = 'mean'
tvfc_estimation_method = 'SVWP_joint'
subject_measures_subset = 'cognitive'

subject_measures_list = hcp_cfg[f"subject-measures-{subject_measures_subset:s}"]
if subject_measures_subset == 'cognitive':
    subject_measures_list = hcp_cfg['subject-measures-nuisance-variables'] + subject_measures_list

scores_savedir = os.path.join(
    hcp_cfg['git-results-basedir'], 'subject_measure_prediction',
    'out_of_sample_prediction', subject_measures_subset
)

out_of_sample_prediction_results_df = pd.read_csv(
    os.path.join(
        scores_savedir, f'{metric:s}_linear_ridge_model_prediction_accuracy_TVFC_{tvfc_summary_measure:s}_{tvfc_estimation_method:s}.csv'
    ),
    index_col=0
)  # (n_permutations, n_subjects)
# print('results')
# print(out_of_sample_prediction_results_df)

out_of_sample_prediction_results_df = out_of_sample_prediction_results_df.loc[:, subject_measures_list]
out_of_sample_prediction_results_df = rename_variables_for_plots(out_of_sample_prediction_results_df, axis=0)
# out_of_sample_prediction_results_df = _rename_models_for_plots(hcp_cfg, out_of_sample_prediction_results_df)


_plot_out_of_sample_prediction_scores(
    config_dict=hcp_cfg,
    out_of_sample_prediction_results_df=out_of_sample_prediction_results_df,
    tvfc_summary_measure=tvfc_summary_measure,
    # figures_savedir='/Users/onnokampman/Developer/'
)

In [ ]:
_plot_out_of_sample_prediction_scores_joint(
    config_dict=hcp_cfg,
    scores_savedir=scores_savedir,
    subject_measures_list=subject_measures_list,
    # performance_metric='prediction_accuracy',
    performance_metric='r2_scores',
    tvfc_estimation_methods=hcp_cfg['plot-models'],
    figures_savedir='/Users/onnokampman/Developer/'
)

### Test-retest

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.test_retest.plot_I2C2_scores import _plot_i2c2_bar

In [ ]:
_plot_i2c2_bar(
    config_dict=hcp_cfg,
    metric='correlation',
    figures_savedir="/Users/onnokampman/Developer/"
)

### Imputation benchmark (run locally)

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.imputation_study.plot_LEOO_likelihoods import _plot_raincloud

In [ ]:
experiment_dimensionality = 'multivariate'

all_likelihoods_df = pd.DataFrame()
for model_name in hcp_cfg['plot-models']:
    likelihoods_filename = f'LEOO_{experiment_dimensionality:s}_likelihoods_{model_name:s}.csv'
    test_likelihoods_savedir = os.path.join(hcp_cfg['git-results-basedir'], 'imputation_study')
    test_likelihoods_filepath = os.path.join(test_likelihoods_savedir, likelihoods_filename)
    if os.path.exists(test_likelihoods_filepath):
        likelihoods_df = pd.read_csv(
            test_likelihoods_filepath, index_col=0
        )  # (n_subjects, n_scans)
        likelihoods_array = likelihoods_df.values.reshape(-1)  # (n_subjects * n_scans, 1)

        model_name = model_name.replace('SVWP_joint', 'WP')
        model_name = model_name.replace('DCC_joint', 'DCC-J')
        model_name = model_name.replace('DCC_bivariate_loop', 'DCC-BL')
        model_name = model_name.replace('SW_cross_validated', 'SW-CV')
        model_name = model_name.replace('_', '-')

        all_likelihoods_df[model_name] = likelihoods_array

_plot_raincloud(
    config_dict=hcp_cfg,
    all_likelihoods_df=all_likelihoods_df,
    data_dimensionality='d15',
    experiment_dimensionality='multivariate',
    figure_savedir="/Users/onnokampman/Developer"
)

### Brain state analysis

#### Brain state clusters (Fig. S8)

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.brain_states.plot_brain_states import _plot_brain_state_cluster_centroids

In [ ]:
_plot_brain_state_cluster_centroids(
    config_dict=hcp_cfg,
    # model_name='SVWP_joint',
    # model_name='DCC_joint',
    model_name='SW_cross_validated',
    n_basis_states=3,
    data_dimensionality='d15',
    figures_savedir="/Users/onnokampman/Developer/"
)

#### Brain state clustering inertias

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.brain_states.plot_brain_states import _plot_brain_state_cluster_centroids

In [ ]:
hcp_inertias_df = pd.read_csv(
    os.path.join(hcp_cfg['git-results-basedir'], 'brain_states', 'correlation_inertias_SVWP_joint.csv'),
    index_col=0
)
hcp_inertias_df

In [ ]:
fig, ax = plt.subplots(
    figsize=set_size()
)
sns.lineplot(
    data=hcp_inertias_df,
    markers=True,
    ax=ax
)
ax.set_xticks(hcp_cfg['n-brain-states-list'])
ax.set_xlabel('number of clusters')
ax.set_ylabel('inertia')
ax.legend(
    title="scan",
)

In [ ]:
fig.savefig(
    '../figures/hcp/brain_states/inertias_SVWP_joint.pdf',
    format='pdf',
    bbox_inches='tight'
)

In [ ]:
model_name = 'SVWP_joint'
n_brain_states = 3

_plot_brain_state_cluster_centroids(
    config_dict=hcp_cfg,
    model_name=model_name,
    n_basis_states=n_brain_states,
    data_dimensionality=data_dimensionality
)

#### Brain state switch rate

In [ ]:
from experiments.resting_state.HCP_PTN1200_recon2.brain_states.plot_brain_state_switch_counts import _plot_brain_state_switch_count

In [ ]:
_plot_brain_state_switch_count(
    config_dict=hcp_cfg,
    n_basis_states=3,
    figure_savedir="/Users/onnokampman/Developer/"
)

### Kernel lengthscales and optimal window length

In [ ]:
from helpers.kernel_analysis import plot_lengthscale_window_length_relation

In [ ]:
kernel_params_savedir = os.path.join(
    hcp_cfg['git-results-basedir'], 'kernel_analysis'
)
kernel_params_df = pd.read_csv(
    os.path.join(kernel_params_savedir, f'{kernel_param:s}_kernel_params.csv'),
    index_col=0
)
print(kernel_params_df)
optimal_window_lengths_df = pd.read_csv(
    os.path.join(hcp_cfg['git-results-basedir'], 'optimal_window_lengths', data_split, 'optimal_window_lengths.csv'),
    index_col=0
)
print(optimal_window_lengths_df)

# Prepare data for plot.
assert kernel_params_df.shape == optimal_window_lengths_df.shape
kernel_params_array = kernel_params_df.values.reshape(-1, 1)
optimal_window_lengths_array = optimal_window_lengths_df.values.reshape(-1, 1)

In [ ]:
plot_lengthscale_window_length_relation(
    hcp_cfg,
    kernel_params_array,
    optimal_window_lengths_array,
    figures_savedir="/Users/onnokampman/Developer/"
)

### The end